<a href="https://colab.research.google.com/github/ktravis213/AirborneDataTools/blob/main/AirborneDataTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Getting started with airborne data from the NASA ASIA-AQ field campaign
# Author: Katie Travis (katherine.travis@nasa.gov)
# Created: 4/12/2024
# Updates:


In [4]:
# Mount Google Drive. To make this work for your own drive, create a folder in the top level called ASIA-AQDC8, and put the aircraft files there from 2/13.
# Get the files from here: https://www-air.larc.nasa.gov/cgi-bin/ArcView/asiaaq?DC8=1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# Christoph Knote made a nice reader for data in ICARTT format.
# ICARTT files are csv files but with specific requirements for NASA data. The header of an ICARTT file has information
# including the PI, instrument, contact information, data usage instructions etc.  Open an ICARTT file in TextEdit or similar
# application and read the header before using the data.
#
#!pip install icartt # do this the first time to get the icartt package, then comment out
import icartt # reader for aircraft data
import pandas as pd #python package for making easy to use data structures
from datetime import datetime, timedelta # for working with time variables
import os # for opening files
import geopandas as gpd # package for geo-referencing data for plotting on a map
import matplotlib.pyplot as plt # plotting package

In [8]:
# Directory where my ASIA-AQ DC-8 files are for the Philippines
os.listdir('/content/drive/MyDrive/ASIA-AQDC8/')

['ASIAAQ-MetNav_DC8_20240206_RA.ict',
 'asiaaq-MMS-1Hz_DC8_20240206_RA.ict',
 'asiaaq-MMS-20Hz_DC8_20240206_RA.ict',
 'ASIAAQ-KCCN-CPC_DC8_20240206_RA.ict',
 'ASIAAQ-KCCN-CCN_DC8_20240206_RA.ict',
 'ASIAAQ-KCCN-SMPS_DC8_20240206_RA.ict',
 'asiaaq-ISAF-CH2O_DC8_20240206_RA.ict',
 'ASIAAQ-CAFS-JV_DC8_20240206_RA.ict',
 'asiaaq-MMS-20Hz_DC8_20240207_RA.ict',
 'ASIAAQ-KCCN-SMPS_DC8_20240207_RA.ict',
 'asiaaq-LGR-NH3_DC8_20240206_RA.ict',
 'ASIAAQ-LI7000-CO2_DC8_20240206_RA.ict',
 'ASIAAQ-DACOM_DC8_20240206_RA.ict',
 'asiaaq-CANOE-NO2_DC8_20240206_RB.ict',
 'asiaaq-CIT-H2O2_DC8_20240206_RA.ict',
 'asiaaq-CIT-SO2_DC8_20240206_RA.ict',
 'asiaaq-CIT-HNO3_DC8_20240206_RA.ict',
 'asiaaq-CIT-HCN_DC8_20240206_RA.ict',
 'asiaaq-CIT-PAA_DC8_20240206_RA.ict',
 'asiaaq-CIT-ISOPN_DC8_20240206_RA.ict',
 'asiaaq-MMS-1Hz_DC8_20240207_RA.ict',
 'asiaaq-ISAF-CH2O_DC8_20240207_RA.ict',
 'ASIAAQ-KSP2-SP2_DC8_20240206_RA.ict',
 'ASIAAQ-K-SP2-D_DC8_20240206_RA.ict',
 'ASIAAQ-CAFS-JV_DC8_20240207_RA.ict',
 'ASIA

In [16]:
# Lets look at PM2.5 along the flight track in the Philippines on 2/13th.  We need the PM2.5 data from one of the instruments and the aircraft position data.
f1='/content/drive/MyDrive/ASIA-AQDC8/ASIAAQ-AMS_DC8_20240213_RA.ict'
f2='/content/drive/MyDrive/ASIA-AQDC8/ASIAAQ-LARGE-SP2_DC8_20240213_RA.ict'
f3='/content/drive/MyDrive/ASIA-AQDC8/ASIAAQ-MetNav_DC8_20240213_RA.ict'

# Read in AMS data (SO4, NO3, NH4, OA)
ict = icartt.Dataset(f1)
ams = ict.data[:]# get data out of dataset
ams = pd.DataFrame(ams)# make into a Pandas dataframe for easy manipulation

# Read in SP2 data (BC)
ict = icartt.Dataset(f2)
sp2 = ict.data[:]
sp2 = pd.DataFrame(sp2)
sp2['Time_Start'] = sp2['Time_Mid'] # Give the sp2 data a Time_Start variable for easy merging

# Read in position data (lon, lat, alt, etc)
ict = icartt.Dataset(f3)
mms= ict.data[:]
mms= pd.DataFrame(mms)

/usr/local/lib/python3.10/dist-packages/icartt/dataset.py:290: UserWarning: Normal comments: required keyword PLATFORM is missing.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/icartt/dataset.py:692: UserWarning: Number of header lines suggested in line 1 (75) do not match actual header lines read (76)
  warnings.warn(


In [18]:
# For now, assume that the data is perfectly time-aligned.
# Combine the ams data with the position data. The AMS actually reports gps data, but the final merge will not use this.
merged_df = pd.merge(ams,mms, on='Time_Start')
merged_df = pd.merge(merged_df,sp2, on='Time_Start')
merged_df

,Time_Start,Time_Stop,Time_Mid_x,LAT_AMS,LON_AMS,ALT_AMS,OA_PM1_AMS,OA_prec_PM1_AMS,OA_DL_PM1_AMS,Sulfate_PM1_AMS,...,Aircraft_Sun_Elevation,Sun_Azimuth,Aircraft_Sun_Azimuth,Mixing_Ratio,Part_Press_Water_Vapor,Sat_Vapor_Press_H2O,Sat_Vapor_Press_Ice,Relative_Humidity,Time_Mid_y,BC_mass
0,32892.0,32893.00,32892.50,15.2130,120.6340,1914.2,-0.1484,0.2229,0.7485,0.0342,...,10.9,-107.6,67.2,10.80,13.93,16.14,18.50,86.31,32892.0,5.5
1,32893.0,32894.00,32893.50,15.2121,120.6339,1914.1,-0.2218,0.2230,0.7485,0.1707,...,11.1,-107.6,67.0,10.80,13.93,16.41,18.84,84.92,32893.0,7.3
2,32894.0,32895.00,32894.50,15.2111,120.6337,1914.2,-0.3139,0.2225,0.7486,0.0662,...,11.3,-107.6,66.8,10.80,13.93,16.41,18.84,84.92,32894.0,0.0
3,32895.0,32896.00,32895.50,15.2101,120.6336,1914.3,-0.3557,0.2220,0.7486,0.2199,...,11.3,-107.6,66.7,10.80,13.92,16.41,18.84,84.86,32895.0,14.2
4,32896.0,32896.99,32896.50,15.2091,120.6334,1914.4,-0.7266,0.2208,0.7486,0.0798,...,11.2,-107.6,66.6,10.76,13.88,16.14,18.50,85.97,32896.0,0.0
5,32898.0,32899.00,32898.50,15.2071,120.6331,1914.4,-0.0403,0.2236,0.7487,0.3194,...,11.4,-107.6,66.3,10.73,13.83,16.14,18.50,85.68,32898.0,17.1
6,32899.0,32900.00,32899.50,15.2061,120.6330,1914.4,-0.0876,0.2237,0.7487,0.1069,...,11.3,-107.6,66.2,10.70,13.80,16.14,18.50,85.46,32899.0,0.0
7,32900.0,32900.99,32900.49,15.2052,120.6329,1914.4,-0.3429,0.2227,0.7488,0.1042,...,11.2,-107.6,66.1,10.67,13.76,16.14,18.50,85.23,32900.0,0.0
8,32904.0,32905.00,32904.50,15.2012,120.6322,1913.8,0.0917,0.2250,0.7489,0.1773,...,11.0,-107.6,65.6,10.71,13.81,16.14,18.50,85.57,32904.0,1.8
9,32905.0,32906.00,32905.50,15.2003,120.6321,1913.6,-0.4509,0.2224,0.7489,0.0737,...,11.0,-107.6,65.4,10.72,13.82,16.14,18.50,85.63,32905.0,5.2


In [21]:
merged_df.columns # get all variable names we need

Index(['Time_Start', 'Time_Stop', 'Time_Mid_x', 'LAT_AMS', 'LON_AMS',
       'ALT_AMS', 'OA_PM1_AMS', 'OA_prec_PM1_AMS', 'OA_DL_PM1_AMS',
       'Sulfate_PM1_AMS', 'Sulfate_prec_PM1_AMS', 'Sulfate_DL_PM1_AMS',
       'Nitrate_PM1_AMS', 'Nitrate_prec_PM1_AMS', 'Nitrate_DL_PM1_AMS',
       'Ammonium_PM1_AMS', 'Ammonium_prec_PM1_AMS', 'Ammonium_DL_PM1_AMS',
       'NR_Chloride_PM1_AMS', 'NR_Chloride_prec_PM1_AMS',
       'NR_Chloride_DL_PM1_AMS', 'Seasalt_PM1_AMS', 'Seasalt_prec_PM1_AMS',
       'Seasalt_DL_PM1_AMS', 'MSA_PM1_AMS', 'MSA_prec_PM1_AMS',
       'MSA_DL_PM1_AMS', 'ClO4_PM1_AMS', 'ClO4_prec_PM1_AMS',
       'ClO4_DL_PM1_AMS', 'Iodine_PM1_AMS', 'Iodine_prec_PM1_AMS',
       'Iodine_DL_PM1_AMS', 'Bromine_PM1_AMS', 'Bromine_prec_PM1_AMS',
       'Bromine_DL_PM1_AMS', 'StdtoVol_AMS', 'CloudFlag_AMS', 'SDDataFlag_AMS',
       'InletRH_AMS', 'Flowrate_AMS', 'AmmBalance_PM1_AMS', 'Density_PM1_AMS',
       'OADensity_PM1_AMS', 'OtoC_Ratio_PM1_AMS', 'HtoC_Ratio_PM1_AMS',
       'OAtoOC

In [22]:
# The AMS observes almost all the components of PM1, which should be very close to PM2.5. The only thing it does not have is BC which we get from the LARGE SP2.
# Let's not include seasalt, chloride, etc right now until we get a handle on the main components of OA + SO4 + NO3 + NH4 + BC
# From the header of each file, units of AMS are in ug/sm3 (273 K & 1013 mb), units of SP2 are in ng/sm3 (0 deg C, 1013.25 mb)
merged_df['PM1'] = merged_df['OA_PM1_AMS'] + merged_df['Sulfate_PM1_AMS'] + merged_df['Ammonium_PM1_AMS'] + merged_df['Nitrate_PM1_AMS'] + merged_df['BC_mass']/1E3

In [24]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(merged_df['Longitude'], merged_df['Latitude']))
# Set the coordinate reference system (CRS) to WGS84 (lat/lon)
gdf.crs = 'EPSG:4326'

In [ ]:
# Set a colormap for the data
cmap = plt.get_cmap('viridis')

# Set up a plot
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])  # Equal width for both subplots
# First subplot for the geographic data
ax1 = fig.add_subplot(gs[0])

# Use normalization for consistent coloring
subset.plot(ax=ax1, column='PM1', cmap=cmap, legend=True, norm=norm, legend_kwds={'label': "PM1, ug m-3", 'shrink': 0.5})
# Add basemap
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
ctx.add_basemap(ax1, crs=subset.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

    # Second subplot for altitude vs. time
    ax2 = fig.add_subplot(gs[1])
    # Assuming 'Time_Start' is convertible to a suitable format for plotting and 'Pressure_Altitude' is your altitude data
    ax2.plot(subset['Time_Start']/60/60+8, subset['Pressure_Altitude']/1E3, color='black')
    ax2.set_xlabel('Local Time', fontsize=ft)
    ax2.set_ylabel('Altitude, kft', fontsize=ft)
    ax2.set_title('Altitude as a Function of Time', fontsize=ft)
    ax2.set_xlim(gdf['Time_Start'][0]/60/60+8, gdf['Time_Start'].max()/60/60+8)
    ax2.tick_params(axis='both', which='major', labelsize=ft)


    plt.subplots_adjust(wspace=0.3)  # Adjust horizontal space between plots
    plt.tight_layout()

    # Save each frame
    plt.savefig(f'Frames/frame_{cc}.png')
    plt.close()
    cc=cc+1

import subprocess

# Change to your desired directory path
desired_path = '/Users/ktravis1/Library/CloudStorage/OneDrive-NASA/ASIA-AQ/Frames/'
os.chdir(desired_path)

# Verify the current working directory
print("Current Working Directory: ", os.getcwd())

subprocess.run([
    'ffmpeg', '-framerate', '10', '-i', 'frame_%d.png',
    '-c:v', 'libx264', '-r', '30', '-pix_fmt', 'yuv420p', 'out.mp4'
])


# Just make a map
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the data, color by altitude
gdf.plot(ax=ax, column='Pressure_Altitude', cmap='viridis', legend=True, legend_kwds={'label': "Altitude"})

# Add a basemap of the Manila region
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Zoom in on Manila region - you might need to adjust these bounds for your specific area of interest
xmin, ymin, xmax, ymax = [gdf.total_bounds[0] - 5000, gdf.total_bounds[1] - 5000, gdf.total_bounds[2] + 5000, gdf.total_bounds[3] + 5000]
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

plt.show()

from matplotlib.path import Path
import matplotlib.patches as patches
import contextily as ctx
import numpy as np
from shapely.geometry import Point
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib import gridspec
